In [16]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from entsoe import EntsoePandasClient
from pandas.tseries.frequencies import to_offset

In [2]:
plotly_config = dict(
    autosize=True,
    #showlegend=False,
    margin=dict(l=0, r=0, b=0, t=0, pad=0),
)

In [34]:
_export_filename = 'entso_e-ukraine-exports'
_import_filename = 'entso_e-ukraine-imports'

# Ukraine electricity net energy flows

See [github.com/EnergieID/entsoe-py](https://github.com/EnergieID/entsoe-py)  and [transparency.entsoe.eu](https://transparency.entsoe.eu/transmission-domain/physicalFlow/show?name=&defaultValue=false&viewType=GRAPH&areaType=BORDER_CTY&atch=false&dateTime.dateTime=01.01.2023+00:00|CET|DAY&border.values=CTY|10Y1001C--00003F!CTY_CTY|10Y1001C--00003F_CTY_CTY|BY&dateTime.timezone=CET_CEST&dateTime.timezone_input=CET+(UTC+1)+/+CEST+(UTC+2))

Thanks to How-To from [thesmartinsights.com: How to query data from the ENTSO-E transparency platform using Python](https://thesmartinsights.com/how-to-query-data-from-the-entso-e-transparency-platform-using-python/)

---

# Obtaining data

### Enter your ENTSO-E API key here
Will later be read from `api_key.json`.

In [3]:
#API_KEY = '<YOUR_KEY_HERE>'
#with open('api_key.json', 'w') as f:
#    json.dump(API_KEY, f)

In [4]:
with open('api_key.json', 'r') as f:
    API_KEY = json.load(f)

In [5]:
client = EntsoePandasClient(api_key=API_KEY)

In [6]:
start = pd.Timestamp('2022-02-01', tz ='UTC')
end = pd.Timestamp('2023-04-01', tz ='UTC')

In [36]:
UKRAINE_CODE = 'UA'
PARTNER_CODES = [
    # 'BY', # Belarus (gives NoMatchingDataError)
    'HU', # Hungary
    'MD', # Moldova
    'PL', # Poland
    'RO', # Romania
    # 'RU', # Russia (gives NoMatchingDataError)
    'SK', # Slovakia
]

In [37]:
ukraine_exports = pd.DataFrame(columns=PARTNER_CODES)

In [38]:
for code in PARTNER_CODES:
    print(code)
    ukraine_exports[code] = client.query_crossborder_flows(UKRAINE_CODE, code, start=start,end=end)

HU
MD
PL
RO
SK


In [39]:
ukraine_imports = pd.DataFrame(columns=PARTNER_CODES)

In [40]:
for code in PARTNER_CODES:
    print(code)
    ukraine_imports[code] = client.query_crossborder_flows(code, UKRAINE_CODE, start=start,end=end)

HU
MD
PL
RO
SK


In [41]:
# Export files to usable formats
# Need to remove timezone info for Excel export:
# ValueError: Excel does not support datetimes with timezones. Please ensure that datetimes are timezone unaware before writing to Excel.
# https://stackoverflow.com/questions/61802080/excelwriter-valueerror-excel-does-not-support-datetime-with-timezone-when-savin
ukraine_exports.set_index(ukraine_exports.index.tz_localize(None)).to_excel(f'{_export_filename}.xlsx')
ukraine_exports.to_csv(f'{_export_filename}.csv')
ukraine_imports.set_index(ukraine_imports.index.tz_localize(None)).to_excel(f'{_import_filename}.xlsx')
ukraine_imports.to_csv(f'{_import_filename}.csv')

In [59]:
# Fill empty values with 0
ukraine_exports = ukraine_exports.fillna(0)
ukraine_imports = ukraine_imports.fillna(0)

In [60]:
pd.to_pickle(ukraine_exports, f'{_export_filename}.pickle')
pd.to_pickle(ukraine_imports, f'{_import_filename}.pickle')

# Analysis

In [61]:
ukraine_exports = pd.read_pickle(f'{_export_filename}.pickle')
ukraine_imports = pd.read_pickle(f'{_import_filename}.pickle')

In [65]:
ukraine_net = ukraine_exports - ukraine_imports

### Resampling to middle of month
See [stackoverflow: pandas monthly resample 15th day](https://stackoverflow.com/questions/47246384/pandas-monthly-resample-15th-day)

```python
df1 = df.resample('MS', loffset=pd.Timedelta(14, 'd')).sum()
```

Warnings: `FutureWarning: 'loffset' in .resample() and in Grouper() is deprecated.`

```python
df.resample(freq="3s", loffset="8H")
```

becomes:

```python
from pandas.tseries.frequencies import to_offset
df = df.resample(freq="3s").mean()
df.index = df.index.to_timestamp() + to_offset("8H")
```

In [66]:
fig = px.bar(
    pd.DataFrame({
        'Net flow': ukraine_net.apply(sum, axis=1).resample('MS').sum(),
    }),
    #title='Net electricity flows of Ukraine to neighboring countries',
    labels={'value': 'Electricity flow in MWh', 'variable': ''},
    #category_orders={'variable': ['Net flows']},
)
fig.update_layout(**plotly_config)
fig.add_annotation( # add a text callout with arrow
    text="10.10.22: Campaign against energy infrastructure begins",
    x=pd.to_datetime('2022-10-10'), y=1.5 * 10e4,
    arrowhead=1, showarrow=True
)
fig.update_layout(showlegend=False)

In [79]:
fig = px.bar(
    pd.DataFrame({
        'Imports': ukraine_imports.apply(lambda x: sum(-x), axis=1),
        'Exports': ukraine_exports.apply(sum, axis=1),
    }).resample('MS').sum(),
    labels={
        'value': 'Monthly electricity flow in MWh',
        'variable': 'Flow direction',
        'index': '',
    },
    # category_orders={'Flow direction': ['Exports', 'Imports']},
)
fig.update_layout(**plotly_config)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="center",
    x=0.9,
))
fig.add_trace(
    go.Scatter(
        y=ukraine_net.apply(sum, axis=1).resample('MS').sum(),
        x=ukraine_net.apply(sum, axis=1).resample('MS').sum().index,
        name='Net'
    )
)
fig.add_annotation( # add a text callout with arrow
    text="10.10.22: Campaign against energy infrastructure begins",
    x=pd.to_datetime('2022-10-10'),
    y=(-1) * 10e4,
    axref="x domain",
    ayref="y",
    ax=-100,
    ay=(-4.6) * 10e4,
    arrowhead=1,
    showarrow=True,
    #bordercolor="#c7c7c7",
    #borderwidth=2,
    #borderpad=4,
    #bgcolor="#ff7f0e",
    #opacity=0.8
)
_filename = 'entso_e-ukraine-electricity-combined_monthly'
fig.write_image(f'{_filename}.svg')
fig.show()

In [91]:
fig = px.bar(
    pd.DataFrame({
        'Imports': ukraine_imports.apply(lambda x: sum(-x), axis=1),
        'Exports': ukraine_exports.apply(sum, axis=1),
    }).resample('1w').sum(),
    labels={
        'value': 'Weekly electricity flow in MWh',
        'variable': 'Flow direction',
        'index': '',
    },
    # category_orders={'Flow direction': ['Exports', 'Imports']},
)
fig.update_layout(**plotly_config)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="center",
    x=0.68,
))
fig.add_trace(
    go.Scatter(
        y=ukraine_net.apply(sum, axis=1).resample('1w').sum(),
        x=ukraine_net.apply(sum, axis=1).resample('1w').sum().index,
        name='Net',
        line_color='#313131',
        line_width=2,
    ),
)
fig.add_annotation( # add a text callout with arrow
    text="10.10.22: Campaign against energy infrastructure begins",
    x=pd.to_datetime('2022-10-10'),
    y=0,
    axref="x domain",
    ayref="y",
    ax=-100,
    ay=(-1.2) * 10e4,
    arrowhead=1,
    showarrow=True,
    #bordercolor="#c7c7c7",
    #borderwidth=2,
    #borderpad=4,
    #bgcolor="#ff7f0e",
    #opacity=0.8
)
fig.add_vline(x=pd.to_datetime('2022-02-24'), line_width=3, line_dash="dash", line_color="purple")
_filename = 'entso_e-ukraine-electricity-combined_weekly'
fig.write_image(f'{_filename}.svg')
fig.write_image(f'{_filename}.png')
fig.show()

In [68]:
px.line(ukraine_exports.resample('1w').sum())

In [69]:
px.line(ukraine_imports.resample('1w').sum())

In [70]:
px.line(ukraine_net.resample('1w').sum())